In [47]:
import re # searching patterns
import pandas as pd # dataframes
import numpy as np # vectors and arrays
import plotly.express as px # interactive plots
import time # to slow down computations for testing
from matplotlib.lines import Line2D # for hand-made legend objects
import matplotlib.patches as patches # figure patches
import matplotlib.pyplot as plt # matplotlib plotting
from plotly.subplots import make_subplots # plotly subplots .. like gridspec
import plotly.graph_objects as go # interactive plots

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# all SRA records for plants were downloaded from NCBI

In [48]:
# loading the data
allsra = pd.read_csv('./sra_results_20210405.csv', sep = ',')
allsra

,Experiment Accession,Experiment Title,Organism Name,Instrument,Submitter,Study Accession,Study Title,Sample Accession,Sample Title,"Total Size, Mb",Total RUNs,Total Spots,Total Bases,Library Name,Library Strategy,Library Source,Library Selection
0,SRX10508740,LJ_WGS_data,Oryza sativa,BGISEQ-500,Chinese Academy of Science,SRP313315,Raw sequencing data for 32 rice genomes,SRS8632875,NaN,7483.75,1,80864864.0,1.212973e+10,LJ_WGS_data,WGS,GENOMIC,RANDOM PCR
1,SRX10508739,ZH11_WGS_data_4,Oryza sativa,BGISEQ-500,Chinese Academy of Science,SRP313315,Raw sequencing data for 32 rice genomes,SRS8632874,NaN,1515.13,1,25618470.0,2.561393e+09,ZH11_WGS_data_4,WGS,GENOMIC,RANDOM PCR
2,SRX10508738,ZH11_WGS_data_3,Oryza sativa,BGISEQ-500,Chinese Academy of Science,SRP313315,Raw sequencing data for 32 rice genomes,SRS8632874,NaN,2430.72,1,41175640.0,4.116844e+09,ZH11_WGS_data_3,WGS,GENOMIC,RANDOM PCR
3,SRX10508737,ZH11_WGS_data_2,Oryza sativa,BGISEQ-500,Chinese Academy of Science,SRP313315,Raw sequencing data for 32 rice genomes,SRS8632874,NaN,2231.19,1,37332582.0,3.732591e+09,ZH11_WGS_data_2,WGS,GENOMIC,RANDOM PCR
4,SRX10508736,ZH11_WGS_data_1,Oryza sativa,BGISEQ-500,Chinese Academy of Science,SRP313315,Raw sequencing data for 32 rice genomes,SRS8632874,NaN,2066.25,1,35034494.0,3.502842e+09,ZH11_WGS_data_1,WGS,GENOMIC,RANDOM PCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027257,SRX001919,NaN,Vigna radiata,454 GS FLX,BIOTEC,SRP000414,Vigna radiata strain:Kamphaeng Saen Genome seq...,SRS001831,NaN,295.20,1,470024.0,1.221423e+08,VRKPS,WGS,GENOMIC,RANDOM
1027258,SRX001880,454 sequencing of Senecio madagascariensis fra...,Senecio madagascariensis,454 GS FLX,University of Queens,SRP000400,Senecio madagascariensis transcriptome project,SRS001794,NaN,51.21,1,81333.0,2.206906e+07,library 4,ChIP-Seq,TRANSCRIPTOMIC,cDNA
1027259,SRX001879,454 sequencing of Sencecio madagascariensis tr...,Senecio madagascariensis,454 GS FLX,University of Queens,SRP000400,Senecio madagascariensis transcriptome project,SRS001793,NaN,47.50,1,76769.0,2.058598e+07,library 3,EST,TRANSCRIPTOMIC,cDNA
1027260,SRX001878,454 sequencing of Sencecio madagascariensis tr...,Senecio madagascariensis,454 GS FLX,University of Queens,SRP000400,Senecio madagascariensis transcriptome project,SRS001793,NaN,16.41,1,26896.0,6.887973e+06,library 2,EST,TRANSCRIPTOMIC,cDNA


In [49]:
# checking out which info is available from the data
allsra.columns

Index(['Experiment Accession', 'Experiment Title', 'Organism Name',
       'Instrument', 'Submitter', 'Study Accession', 'Study Title',
       'Sample Accession', 'Sample Title', 'Total Size, Mb', 'Total RUNs',
       'Total Spots', 'Total Bases', 'Library Name', 'Library Strategy',
       'Library Source', 'Library Selection'],
      dtype='object')

In [50]:
# how to start filtering out data
# check out unique types of library source
allsra['Library Source'].unique()

array(['GENOMIC', 'TRANSCRIPTOMIC', 'METAGENOMIC', 'OTHER',
       'METATRANSCRIPTOMIC', 'SYNTHETIC', 'GENOMIC SINGLE CELL',
       'TRANSCRIPTOMIC SINGLE CELL', 'VIRAL RNA'], dtype=object)

In [51]:
# explore further the 'other'. Do titles sound like something we want?
allsra[allsra['Library Source'] == 'OTHER']['Study Title'].unique() # looks too technicall for what we need

array(['Quantitative DNA metabarcoding experiment',
       'Study of the fungal bacterial and archaeal diversity associated with Triticum aestivum var. Paragon roots',
       'Camellia sinensis var. sinensis cultivar:Tieguanyin Genome sequencing and assembly',
       'Petunia hybrida DAP-seq raw sequence reads',
       'Deep sequencing of small RNA fraction isolated from raspberry and Ribes sp.',
       'ChIP-seq of H3K27me3 in rice emf2a mutant',
       'transcriptome and H3K27me3 ChIP-seq in WT and emf2a mutant',
       'Histone H1 is required for proper genome folding and coupling DNA methylation with gene expression',
       'MAGO proteins safeguard the plant male germline under environmental stress',
       'Aconitum kusnezoffii Raw sequence reads',
       'Euonymus alatus Raw sequence reads',
       'Pisum sativum Raw sequence reads',
       'Glycyrrhiza uralensis Raw sequence reads',
       'The complete chloroplast genome sequence of Actinidia arguta',
       'complete chloropl

In [52]:
# filtering and limiting to 'genomic'
genomesra = allsra[allsra['Library Source'] == 'GENOMIC']
# print length of filtered dataframe
print(len(genomesra))
# unique types of library strategy
genomesra['Library Strategy'].unique()

735364


array(['WGS', 'Targeted-Capture', 'Hi-C', 'OTHER', 'RAD-Seq', 'RNA-Seq',
       'ChIP-Seq', 'GBS', 'WGA', 'WXS', 'Bisulfite-Seq', 'AMPLICON',
       'DNase-Hypersensitivity', 'MRE-Seq', 'MNase-Seq', 'ATAC-seq',
       'MBD-Seq', 'FL-cDNA', 'RIP-Seq', 'miRNA-Seq', 'CLONE',
       'Tethered Chromatin Conformation Capture', 'WCS', 'FAIRE-seq',
       'Synthetic-Long-Read', 'MeDIP-Seq', 'CLONEEND', 'POOLCLONE',
       'FINISHING', 'EST', 'Tn-Seq', 'ChIA-PET', 'ChIP', 'other'],
      dtype=object)

In [53]:
# strategy = ['WGS', 'Targeted-Capture', 'OTHER', 'RAD-Seq', 'ChIP-Seq', 'GBS', 'WGA','WXS','AMPLICON','WCS','other']
# filtering and limiting to target sequence data
strategy = ['Targeted-Capture']
# create new filtered dataframe
genomesra1 = genomesra[genomesra['Library Strategy'].isin(strategy)]
genomesra1

,Experiment Accession,Experiment Title,Organism Name,Instrument,Submitter,Study Accession,Study Title,Sample Accession,Sample Title,"Total Size, Mb",Total RUNs,Total Spots,Total Bases,Library Name,Library Strategy,Library Source,Library Selection
118,SRX10508039,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632265,NaN,281.41,1,2125445.0,6.376335e+08,CFC-9,Targeted-Capture,GENOMIC,Hybrid Selection
119,SRX10508038,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632264,NaN,432.05,1,3247805.0,9.743415e+08,CED-3,Targeted-Capture,GENOMIC,Hybrid Selection
120,SRX10508037,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632263,NaN,120.95,1,910689.0,2.732067e+08,SWC-5,Targeted-Capture,GENOMIC,Hybrid Selection
121,SRX10508036,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632262,NaN,189.54,1,1434460.0,4.303380e+08,SPB-11,Targeted-Capture,GENOMIC,Hybrid Selection
122,SRX10508035,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632261,NaN,281.53,1,2105885.0,6.317655e+08,CCCD-98,Targeted-Capture,GENOMIC,Hybrid Selection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876574,ERX1365514,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066749,NaN,862.78,1,3683854.0,1.849295e+09,unspecified,Targeted-Capture,GENOMIC,other
876575,ERX1365513,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066748,NaN,470.01,1,1518349.0,7.890861e+08,unspecified,Targeted-Capture,GENOMIC,other
876576,ERX1365512,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066747,NaN,609.47,1,1814975.0,9.812422e+08,unspecified,Targeted-Capture,GENOMIC,other
876577,ERX1365511,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066746,NaN,616.72,1,2742300.0,1.376635e+09,unspecified,Targeted-Capture,GENOMIC,other


In [54]:
# print unique library selection types
genomesra1['Library Selection'].unique()

array(['Hybrid Selection', 'RANDOM', 'other', 'unspecified',
       'Reduced Representation', 'PCR', 'padlock probes capture method',
       'Restriction Digest', 'size fractionation'], dtype=object)

In [55]:
# limit to the ones in the 'selection' list
selection = ['RANDOM PCR', 'RANDOM', 'Hybrid Selection', 'Reduced Representation', 'Restriction Digest', 'unspecified',
       'PCR', 'DNase','other', 'ChIP', 'cDNA', 'padlock probes capture method','HMPR','MDA']
genomesra2 = genomesra1[genomesra1['Library Selection'].isin(selection)]
genomesra2

,Experiment Accession,Experiment Title,Organism Name,Instrument,Submitter,Study Accession,Study Title,Sample Accession,Sample Title,"Total Size, Mb",Total RUNs,Total Spots,Total Bases,Library Name,Library Strategy,Library Source,Library Selection
118,SRX10508039,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632265,NaN,281.41,1,2125445.0,6.376335e+08,CFC-9,Targeted-Capture,GENOMIC,Hybrid Selection
119,SRX10508038,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632264,NaN,432.05,1,3247805.0,9.743415e+08,CED-3,Targeted-Capture,GENOMIC,Hybrid Selection
120,SRX10508037,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632263,NaN,120.95,1,910689.0,2.732067e+08,SWC-5,Targeted-Capture,GENOMIC,Hybrid Selection
121,SRX10508036,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632262,NaN,189.54,1,1434460.0,4.303380e+08,SPB-11,Targeted-Capture,GENOMIC,Hybrid Selection
122,SRX10508035,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632261,NaN,281.53,1,2105885.0,6.317655e+08,CCCD-98,Targeted-Capture,GENOMIC,Hybrid Selection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876574,ERX1365514,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066749,NaN,862.78,1,3683854.0,1.849295e+09,unspecified,Targeted-Capture,GENOMIC,other
876575,ERX1365513,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066748,NaN,470.01,1,1518349.0,7.890861e+08,unspecified,Targeted-Capture,GENOMIC,other
876576,ERX1365512,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066747,NaN,609.47,1,1814975.0,9.812422e+08,unspecified,Targeted-Capture,GENOMIC,other
876577,ERX1365511,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066746,NaN,616.72,1,2742300.0,1.376635e+09,unspecified,Targeted-Capture,GENOMIC,other


In [57]:
# we want to check how many accessions there are PER species
# grouping the dataframe by organism and library strategy
# grouping by library strategy can be ignored if there's only one type of library strategy
byorglib = genomesra2.groupby(by = ['Organism Name','Library Strategy'])['Experiment Accession'].count().reset_index()
# sort values by number of experiment accessions per species
byorglib.sort_values('Experiment Accession', ascending = False)

,Organism Name,Library Strategy,Experiment Accession
3106,Picea abies,Targeted-Capture,3076
4057,Triticum aestivum,Targeted-Capture,2688
4251,Zea mays,Targeted-Capture,2685
1849,Hordeum vulgare subsp. vulgare,Targeted-Capture,910
3225,Populus tremula,Targeted-Capture,766
...,...,...,...
1609,Funaria sp. Goffinet et al. 9934,Targeted-Capture,1
1610,Funaria sp. Hylander 5947,Targeted-Capture,1
1611,Funaria sp. Hylander 5948,Targeted-Capture,1
1612,Fusaea peruviana,Targeted-Capture,1


In [58]:
# some species names have parenthesis.. check them out
list(byorglib[byorglib['Organism Name'].str.contains('\(')]['Organism Name'].unique())

['Costus sp. Kay 18-001(KK0058)',
 'Costus sp. Kay 18-001(KK0073)',
 'Erythroxylum sp. (D.M. White 530)',
 'Erythroxylum sp. Brewer LA (B.Hyland 13373)',
 'Erythroxylum sp. Cholmondely Creek (J.R.Clarkson 9367)',
 'Erythroxylum sp. Splityard Creek (L.Pedley 5360)',
 'Nepenthes adrianii (nom. nud.)']

In [59]:
# let's remove this list of species:
# most of these appear when filtering the dataframe by other values
# leaving this here for legacy
remove = ['(Arabidopsis thaliana x Arabidopsis arenosa) x Arabidopsis suecica',
 '(Prunus cerasifera x Prunus salicina) x (Prunus cerasifera x Prunus persica)', '(Prunus dulcis x Prunus persica) x (Prunus dulcis x Prunus davidiana)',
 '(Prunus dulcis x Prunus persica) x (Prunus dulcis x Prunus persica)', '(Prunus persica x Prunus davidiana) x (Prunus dulcis x Prunus persica)',
 '(Prunus persica x Prunus davidiana) x Prunus persica', 'Malus domestica x (Pyrus pyrifolia x Pyrus communis)',
          'Prunus cerasifera x (Prunus persica x Prunus dulcis)',
 'Prunus persica x (Prunus dulcis x Prunus persica)',
 'Prunus persica x (Prunus persica x Prunus davidiana)',]

genomesra3 = genomesra2[~genomesra2['Organism Name'].isin(remove)]
# genomesra3

In [60]:
# let's add a genera column to help filtering things we know aren't too useful
genomesra3['genus'] = genomesra3['Organism Name'].replace('([A-Za-z]+) .+','\\1', regex = True).str.replace('[^a-zA-Z \d]','')
genomesra3['genus'].unique()

array(['Castilleja', 'Geonoma', 'Veronica', 'Aiphanes', 'Arabidopsis',
       'Brassica', 'Picea', 'Pavonia', 'Hibiscus', 'x', 'Asterogyne',
       'Ceroxylon', 'Kerriodoxa', 'Nypa', 'Calamus', 'Plectocomiinae',
       'Plectocomiopsis', 'Myrialepis', 'Plectocomia', 'Pigafetta',
       'Metroxylon', 'Salacca', 'Eleiodoxa', 'Korthalsia', 'Mauritiella',
       'Mauritia', 'Lepidocaryum', 'Raphia', 'Laccosperma', 'Eremospatha',
       'Oncocalamus', 'Eugeissona', 'Rhynchospora', 'Cardamine',
       'Capurodendron', 'Bemangidia', 'Mimusops', 'Faucherea',
       'Neolemonniera', 'Lecomtedoxa', 'Gluema', 'Inhambanella',
       'Donella', 'Tsebona', 'Isonandra', 'Madhuca', 'Diploknema',
       'Sideroxylon', 'Begonia', 'Cymbidium', 'Lycomormium', 'Tolumnia',
       'Stanhopea', 'Catasetum', 'Gastrorchis', 'Coelogyne',
       'Paphiopedilum', 'Phragmipedium', 'Spiranthes', 'Calopogon',
       'Acineta', 'Maxillaria', 'Anoectochilus', 'Polystachya',
       'Angraecum', 'Phalaenopsis', 'Platanth

In [61]:
# let's group by genus instead and filter out genera with 1-2 experiment accessions
byorglib = genomesra3.groupby(by = ['genus'])['Experiment Accession'].count().reset_index()
byorglib.sort_values('Experiment Accession', ascending = False, inplace = True)

# make a list of species with less than or equal to 2 experiment accessions per genus
remove = list(byorglib[byorglib['Experiment Accession'] <= 2]['genus'].unique())
# list of remaining species with more than 2 expriment accessions
remain = byorglib[byorglib['Experiment Accession'] > 2]['genus'].unique()
print('genera with two or less experiment accessions: ',len(remove))
print('genera with three or more experiment accessions: ',len(remain))

genera with two or less experiment accessions:  740
genera with three or more experiment accessions:  226


In [62]:
# just checking that there is no Arabidopsis records
[x for x in remove if 'Ara' in str(x)]

['Aralia']

In [63]:
# now, removing the genera we flagged above
genomesra4 = genomesra3[~genomesra3['genus'].isin(remove)] # because I added genera to the remove list that have <= 3 accessions
genomesra4

,Experiment Accession,Experiment Title,Organism Name,Instrument,Submitter,Study Accession,Study Title,Sample Accession,Sample Title,"Total Size, Mb",Total RUNs,Total Spots,Total Bases,Library Name,Library Strategy,Library Source,Library Selection,genus
118,SRX10508039,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632265,NaN,281.41,1,2125445.0,6.376335e+08,CFC-9,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja
119,SRX10508038,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632264,NaN,432.05,1,3247805.0,9.743415e+08,CED-3,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja
120,SRX10508037,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632263,NaN,120.95,1,910689.0,2.732067e+08,SWC-5,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja
121,SRX10508036,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632262,NaN,189.54,1,1434460.0,4.303380e+08,SPB-11,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja
122,SRX10508035,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632261,NaN,281.53,1,2105885.0,6.317655e+08,CCCD-98,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876574,ERX1365514,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066749,NaN,862.78,1,3683854.0,1.849295e+09,unspecified,Targeted-Capture,GENOMIC,other,Solanum
876575,ERX1365513,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066748,NaN,470.01,1,1518349.0,7.890861e+08,unspecified,Targeted-Capture,GENOMIC,other,Solanum
876576,ERX1365512,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066747,NaN,609.47,1,1814975.0,9.812422e+08,unspecified,Targeted-Capture,GENOMIC,other,Solanum
876577,ERX1365511,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066746,NaN,616.72,1,2742300.0,1.376635e+09,unspecified,Targeted-Capture,GENOMIC,other,Solanum


In [64]:
# checking the genera again
byorglib = genomesra4.groupby(by = ['genus'])['Experiment Accession'].count().reset_index()
byorglib.sort_values('Experiment Accession', ascending = False, inplace = True)

# print the genera with more than 1500 accessions
byorglib[byorglib['Experiment Accession'] >= 1500]['genus'].values

array(['Picea', 'Triticum', 'Zea'], dtype=object)

In [65]:
remove = ['Oryza', 'Hordeum', 'Arabidopsis', 'Helianthus',
       'Brassica', 'Sorghum',  'Pinus', 'Picea', 'Malus', 'Prunus', 'Cicer', 
       'Eucalyptus','x']
# filter again
genomesra5 = genomesra4[~genomesra4['genus'].isin(remove)]
# groupby againa
byorglib = genomesra5.groupby(by = ['genus'])['Experiment Accession'].count().reset_index()
byorglib.sort_values('Experiment Accession', ascending = False, inplace = True)
byorglib

,genus,Experiment Accession
202,Triticum,2702
210,Zea,2685
160,Populus,1298
113,Lolium,736
87,Geonoma,681
...,...,...
155,Plerandra,3
156,Pleroma,3
161,Prometheum,3
170,Rhabdophyllum,3


In [66]:
fig = make_subplots(rows = 1, cols = 1, column_widths = [50], row_heights = [5])
fig.add_trace(go.Bar(x = byorglib['genus'],
             y = byorglib['Experiment Accession'], textposition = 'outside', textfont = dict(size = 6)),
             row=1, col=1)
fig.show()# not zero indexed ffs

In [67]:
from ete3 import NCBITaxa
# the two lines below need to be run only once
# ncbi = NCBITaxa()
# ncbi.update_taxonomy_database()

# define a function to annotate the taxonomic information

def taxAnnotate(df, scinames=''):
    """
    Uses ete3 to recover the taxonomic hierarchy of unique species from the blast results
    organizes the taxonomic data in hierarchichal dictionaries, then maps them on the dataframe
    it takes a dataframe and a string name for the column with the organims name
    """
    ncbi = NCBITaxa() # activates the ncbi database. Download it first
    spp_list=[x for x in df[scinames].replace('sp.','').str.replace('[^a-zA-Z \d]','').unique()]
    # dictionary of species:ID
    name2taxid = ncbi.get_name_translator(spp_list)
    second={};third={};fourth={}
    for key,value in name2taxid.items():
        lineage = ncbi.get_lineage(value[0])
        # retrieves a dictionary of names:IDs for the taxonomic levels in value
        names = ncbi.get_taxid_translator(lineage)
        # now we will retain the second, third and fifth taxonomic categories in a dictionary
        second[key]=list(names.values())[2] if len(list(names.values())) >1 else 'unassigned'
        third[key]=list(names.values())[4] if len(list(names.values())) >2 else 'unassigned'
        fourth[key]=list(names.values())[5] if len(list(names.values())) >3 else 'unassigned'
        
        df['tax_level1']=df[scinames].map(second)
        df['tax_level2']=df[scinames].map(third)
        df['tax_level3']=df[scinames].map(fourth)
        
    return df

In [68]:
taxAnnotate(genomesra5, scinames = 'Organism Name')
genomesra5

,Experiment Accession,Experiment Title,Organism Name,Instrument,Submitter,Study Accession,Study Title,Sample Accession,Sample Title,"Total Size, Mb",...,Total Spots,Total Bases,Library Name,Library Strategy,Library Source,Library Selection,genus,tax_level1,tax_level2,tax_level3
118,SRX10508039,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632265,NaN,281.41,...,2125445.0,6.376335e+08,CFC-9,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja,Embryophyta,Lamiales,Viridiplantae
119,SRX10508038,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632264,NaN,432.05,...,3247805.0,9.743415e+08,CED-3,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja,Embryophyta,Lamiales,Viridiplantae
120,SRX10508037,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632263,NaN,120.95,...,910689.0,2.732067e+08,SWC-5,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja,Embryophyta,Lamiales,Viridiplantae
121,SRX10508036,Angiosperms 353 target capture of Castilleja s...,Castilleja sessiliflora,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632262,NaN,189.54,...,1434460.0,4.303380e+08,SPB-11,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja,Embryophyta,Lamiales,Viridiplantae
122,SRX10508035,Angiosperms 353 target capture of Castilleja c...,Castilleja citrina,NextSeq 500,Northwestern University,SRP313302,Range-wide sampling of Castilleja sessiliflora...,SRS8632261,NaN,281.53,...,2105885.0,6.317655e+08,CCCD-98,Targeted-Capture,GENOMIC,Hybrid Selection,Castilleja,Embryophyta,Lamiales,Viridiplantae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876574,ERX1365514,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066749,NaN,862.78,...,3683854.0,1.849295e+09,unspecified,Targeted-Capture,GENOMIC,other,Solanum,Embryophyta,Solanales,Solanaceae
876575,ERX1365513,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066748,NaN,470.01,...,1518349.0,7.890861e+08,unspecified,Targeted-Capture,GENOMIC,other,Solanum,Embryophyta,Solanales,Solanaceae
876576,ERX1365512,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066747,NaN,609.47,...,1814975.0,9.812422e+08,unspecified,Targeted-Capture,GENOMIC,other,Solanum,Embryophyta,Solanales,Solanaceae
876577,ERX1365511,Illumina MiSeq paired end sequencing,Solanum okadae,Illumina MiSeq,NaN,ERP014354,Utilizing 'Omics' technologies to identify and...,ERS1066746,NaN,616.72,...,2742300.0,1.376635e+09,unspecified,Targeted-Capture,GENOMIC,other,Solanum,Embryophyta,Solanales,Solanaceae


In [69]:
# now, let's groupby using family
byorglib = genomesra5.groupby(by = ['tax_level3'])['Experiment Accession'].count().reset_index()
byorglib.sort_values('Experiment Accession', ascending = False, inplace = True)
byorglib

fig = make_subplots(rows = 1, cols = 1, column_widths = [50], row_heights = [5])
fig.add_trace(go.Bar(x = byorglib['tax_level3'],
             y = byorglib['Experiment Accession'], textposition = 'outside', textfont = dict(size = 6)),
             row=1, col=1)
fig.show()# not zero indexed ffs

In [70]:
# now, we can filter by Arecaecae
palms = genomesra5[genomesra5['tax_level3'] == 'Arecaceae']
# groupby genera
byorglib = palms.groupby(by = ['genus'])['Experiment Accession'].count().reset_index()
byorglib.sort_values('Experiment Accession', ascending = False, inplace = True)
byorglib

fig = make_subplots(rows = 1, cols = 1, column_widths = [50], row_heights = [5])
fig.add_trace(go.Bar(x = byorglib['genus'],
             y = byorglib['Experiment Accession'], textposition = 'outside', textfont = dict(size = 6)),
             row=1, col=1)
fig.show()# not zero indexed ffs

In [71]:
# let's imagine we don't have Calyptronoma
SRR = list(palms[palms['genus'] == 'Calyptronoma']['Experiment Accession'].values)
palms[palms['genus'] == 'Calyptronoma'][['genus','Organism Name','Experiment Accession']]

,genus,Organism Name,Experiment Accession
345177,Calyptronoma,Calyptronoma rivalis,SRX5978737
345178,Calyptronoma,Calyptronoma occidentalis,SRX5978736
345837,Calyptronoma,Calyptronoma occidentalis,SRX5964004
345838,Calyptronoma,Calyptronoma rivalis,SRX5964003


In [ ]:
# to download use the sra-toolkit:
# https://github.com/ncbi/sra-tools
%%bash

for i in SRX5978737 SRX5978736 SRX5964004 SRX5964003; do ./bin/fasterq-dump $i; done